In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time
import joblib

#ruta del csv
df = pd.read_csv('test.csv')

# Dividir los datos
X = df.drop('Cancer Present', axis=1)
y = df['Cancer Present']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


Entrenamiento del modelo secuencial

In [3]:

# Entrenar el modelo
modelo_knn = KNeighborsClassifier(n_neighbors=20)
modelo_knn.fit(X_train, y_train)

#tiempo de ejecucion

start_time = time.time()

y_pred_secuencial = modelo_knn.predict(X_test)

end_time = time.time()
total_time = end_time - start_time

print("Tiempo de ejecucion: ", total_time)


Tiempo de ejecucion:  22.015395641326904


In [4]:
evaluacion_secuencial = accuracy_score(y_test, y_pred_secuencial)
print("Evaluacion secuencial: ", evaluacion_secuencial)

Evaluacion secuencial:  0.4995047619047619


Entrenamiento del modelo con procesamiento paralelo

In [5]:
import time
import numpy as np
import joblib
import threading
from sklearn.neighbors import KNeighborsClassifier

# Función wrapper para la predicción
def predict_wrapper(model, data):
    return model.predict(data)


start_time_parallel = time.time()

# Paralelizar la predicción 
with joblib.parallel_backend('threading', n_jobs=-1):
    y_pred_parallel = joblib.Parallel()(joblib.delayed(predict_wrapper)(modelo_knn, X_test_batch) for X_test_batch in np.array_split(X_test, joblib.cpu_count()))

# Concatenar los resultados de la predicción paralela
y_pred_parallel = np.concatenate(y_pred_parallel, axis=0)

end_time_parallel = time.time()

print("Tiempo de ejecución paralelo: ", end_time_parallel - start_time_parallel)


Tiempo de ejecución paralelo:  13.328596115112305


In [6]:
evaluacion_paralela = accuracy_score(y_test, y_pred_parallel)
evaluacion_paralela

0.4995047619047619